# Description

In this notebook we can explore the characteristics of interaction events, such as their lengths, the associated ball displacements, etc.

# Imports

In [ ]:
from pathlib import Path
import sys

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

src_path = Path.cwd().parent / "src"
if src_path not in sys.path:
    sys.path.append(str(src_path))

from Ballpushing_utils import Experiment, Fly

from utils_behavior import Processing

# Generate a Fly object

In [ ]:
ExampleFly_path = Path(
    "/mnt/upramdya_data/MD/MultiMazeRecorder/Videos/231130_TNT_Fine_2_Videos_Tracked/arena2/corridor5"
)

ExampleFly = Fly(ExampleFly_path, as_individual=True)

## Get the list of interaction events using the above config parameters

In [ ]:
interaction_events = ExampleFly.tracking_data.interaction_events[0][0]

interaction_events

In [ ]:
# Take the 3rd element of each interaction event (duration) and average it

interaction_durations = [event[2] for event in interaction_events]

average_interaction_duration = sum(interaction_durations) / len(interaction_durations)

average_duration_time = average_interaction_duration / ExampleFly.experiment.fps

print(f"Average interaction duration: {average_duration_time:.2f} seconds or {average_interaction_duration:.2f} frames")

Here we have an idea of the average duration of an interaction between the fly and the ball. Another interesting metric would be to look at the average duration of events after the event onset, as it's the critical point for UMAP

In [ ]:
interaction_onsets = ExampleFly.tracking_data.interactions_onsets[0, 0]

interaction_onsets

In [ ]:
# Compute the adjusted duration for each event, from the interaction onset to the end of the interaction

adjusted_interaction_durations = []

# Ensure interaction_onsets and interaction_events have the same length
if len(interaction_onsets) != len(interaction_events):
    raise ValueError("Mismatch between the number of interaction onsets and interaction events.")

for i, event in enumerate(interaction_events):
    try:
        start_time = interaction_onsets[i]
        end_time = event[1]
        if start_time is None or end_time is None:
            raise ValueError(f"Invalid start or end time for event {i}.")
        adjusted_duration = end_time - start_time
        adjusted_interaction_durations.append(adjusted_duration)
    except Exception as e:
        print(f"Error processing event {i}: {e}")

# Calculate the average adjusted interaction duration
if adjusted_interaction_durations:
    average_adjusted_interaction_duration = sum(adjusted_interaction_durations) / len(adjusted_interaction_durations)

    average_adjusted_interaction_duration_time = average_adjusted_interaction_duration / ExampleFly.experiment.fps
    print(f"Average adjusted interaction duration: {average_adjusted_interaction_duration_time:.2f} seconds or {average_adjusted_interaction_duration:.2f} frames")
else:
    print("No valid adjusted interaction durations to calculate an average.")

So here we see on average, an event lasts 212 frames. So maximum standardized contact length should be around 200

# Test on full experiment

Flies can be quite different from one another so we're gonna generate a concatenated list of all events for one experiment.

In [ ]:
ExampleExperiment_path = Path(
    "/mnt/upramdya_data/MD/MultiMazeRecorder/Videos/231130_TNT_Fine_2_Videos_Tracked/"
)
ExampleExperiment = Experiment(
    ExampleExperiment_path,
)

In [ ]:
# For each fly, get the adjusted interaction durations
adjusted_interaction_durations_all_flies = []

for fly in ExampleExperiment.flies:
    interaction_events = fly.tracking_data.interaction_events[0][0]
    interaction_onsets = fly.tracking_data.interactions_onsets[0, 0]

    # Ensure interaction_onsets and interaction_events have the same length
    if len(interaction_onsets) != len(interaction_events):
        raise ValueError("Mismatch between the number of interaction onsets and interaction events.")

    for i, event in enumerate(interaction_events):
        try:
            start_time = interaction_onsets[i]
            end_time = event[1]
            if start_time is None or end_time is None:
                raise ValueError(f"Invalid start or end time for event {i}.")
            adjusted_duration = end_time - start_time
            adjusted_interaction_durations.append(adjusted_duration)
        except Exception as e:
            print(f"Error processing event {i}: {e}")

    # Calculate the average adjusted interaction duration
    if adjusted_interaction_durations:
        average_adjusted_interaction_duration = sum(adjusted_interaction_durations) / len(adjusted_interaction_durations)

        average_adjusted_interaction_duration_time = average_adjusted_interaction_duration / ExampleFly.experiment.fps
        print(
            f"Average adjusted interaction duration: {average_adjusted_interaction_duration_time:.2f} seconds or {average_adjusted_interaction_duration:.2f} frames"
        )

        adjusted_interaction_durations_all_flies.append(adjusted_interaction_durations)
    else:
        print("No valid adjusted interaction durations to calculate an average.")

In [ ]:
# Get the average adjusted interaction duration across all flies

if adjusted_interaction_durations_all_flies:
    all_durations_flat = [duration for sublist in adjusted_interaction_durations_all_flies for duration in sublist]
    average_adjusted_interaction_duration_all_flies = sum(all_durations_flat) / len(all_durations_flat)

    average_adjusted_interaction_duration_time_all_flies = average_adjusted_interaction_duration_all_flies / ExampleFly.experiment.fps
    print(
        f"Average adjusted interaction duration across all flies: {average_adjusted_interaction_duration_time_all_flies:.2f} seconds or {average_adjusted_interaction_duration_all_flies:.2f} frames"
    )
else:
    print("No valid adjusted interaction durations to calculate an average across all flies.")

In [ ]:
# Generate an histogram of the adjusted interaction durations

plt.figure(figsize=(10, 6))

sns.histplot(
    adjusted_interaction_durations_all_flies,
    bins=30,
    kde=False,
    color="blue",
)

# Remove legend
plt.legend().remove()

Data is skewed, actually although the average is 200, most interactions are found at much lower value

In [ ]:
# Compute the median of the adjusted interaction durations

median_adjusted_interaction_duration = np.median(adjusted_interaction_durations_all_flies)

print(
    f"Median adjusted interaction duration across all flies: {median_adjusted_interaction_duration:.2f} frames"
)

Let's also generate some bootstrapped confidence interval of the median

In [ ]:
bs_ci_median = Processing.draw_bs_ci(
    adjusted_interaction_durations_all_flies,
    func=np.median,
    n_reps=1000,
)

bs_ci_median

In [ ]:
bs_ci_mean = Processing.draw_bs_ci(
    adjusted_interaction_durations_all_flies,
    func=np.mean,
    n_reps=1000,
)
bs_ci_mean

Here we can see that the bootstrapped interval is super narrow around 115 for median and 210 for mean. Data being very skewed, it is likely that median is a better indicator, so 115 frames makes sense.

In [ ]:
# Redo the histogram with the mean and median as vertical lines

plt.figure(figsize=(10, 6))

sns.histplot(
    adjusted_interaction_durations_all_flies,
    bins=30,
    kde=False,
    color="blue",
)

# Add mean and median lines

plt.axvline(
    x=average_adjusted_interaction_duration_all_flies,
    color="red",
    linestyle="--",
    label=f"Mean: {bs_ci_mean[0]:.2f} frames",
)

plt.axvline(
    x=median_adjusted_interaction_duration,
    color="green",
    linestyle="--",
    label=f"Median: {bs_ci_median[0]:.2f} frames",
)


# Remove legend
plt.legend().remove()